## 1. What is PyTorch?

**Answer:**  
PyTorch is an **open-source deep learning framework** developed by **Facebook’s AI Research Lab (FAIR)**.  
It provides a **flexible and efficient platform** for building, training, and deploying machine learning and deep learning models — especially suited for research because of its **imperative (Pythonic) programming style**.

### 🔑 Key Features
- **Dynamic computation graph (eager execution)** — graphs are built on-the-fly as operations run.  
- **Strong GPU acceleration** via CUDA.  
- **Autograd engine** for automatic differentiation.  
- **`torch.nn` and `torch.optim`** modules for easy model building and optimization.  
- **Seamless integration** with Python libraries like NumPy and SciPy.

---

## 2. Explain PyTorch’s Dynamic Computation Graph and How It Differs from TensorFlow’s Static Graph

### 🔹 Dynamic Graph (PyTorch)
- PyTorch builds the computation graph **dynamically during runtime**.  
- Each operation (e.g., `a = x + y`) immediately creates a node in the graph **and executes it**.  
- You can use **normal Python control flow** (loops, if-statements) inside models.  
- The graph is **rebuilt each forward pass**, making it highly flexible and debuggable.  

### 🔹 Static Graph (TensorFlow 1.x)
- TensorFlow 1.x builds a **static graph before execution**.  
- You first **define the full computation graph symbolically**, then call `session.run()` to execute it.  
- It’s **more optimized for deployment**, but **less flexible for experimentation**.  

---

✅ **Summary Table**

| Framework | Graph Type | When Built | Debugging | Flexibility |
|------------|-------------|------------|------------|--------------|
| **PyTorch** | Dynamic | At runtime | Easy (uses Python debugger) | Very high |
| **TensorFlow 1.x** | Static | Before execution | Harder | Lower |
| **TensorFlow 2.x (Eager Mode)** | Dynamic | Runtime | Easier | Similar to PyTorch |


## Kmeans

✅ Example of placement:

| Expression       | New shape | Description                          |
|------------------|------------|--------------------------------------|
| `X[None, :]`     | `(1, 2, 2)` | adds a new axis at the **front**     |
| `X[:, None]`     | `(2, 1, 2)` | adds a new axis in the **middle**    |
| `X[:, :, None]`  | `(2, 2, 1)` | adds a new axis at the **end**       |

In [2]:
import numpy as np

def kmeans(X, k=3, max_iters=100):
    # Randomly choose k points as initial centroids
    np.random.seed(42)
    centroids = X[np.random.choice(len(X), k, replace=False)]

    for _ in range(max_iters):
        # Step 1: Assign points to nearest centroid
        ## Broadcast [N*d] and [k*d] ==> [N*k*d] - [N*k*d]
        distances = np.linalg.norm(X[:, None] - centroids[None, :], axis=2)
        labels = np.argmin(distances, axis=1)

        # Step 2: Recompute centroids as mean of assigned points
        new_centroids = np.array([X[labels == i].mean(axis=0) for i in range(k)])

        # Stop if centroids do not change
        if np.allclose(centroids, new_centroids):
            break
        centroids = new_centroids

    return centroids, labels

## Gradient Descent

In [ ]:
def gd_linear_regression(X, y, lr, epochs, tol):
    X = np.asarray(X, dtype=float)
    y = np.asarray(y, dtype=float).reshape(-1)

    n,d = X.shape
    Xb = np.c_[np.ones(n), X]
    w = np.zeros(d+1)
    history = [] 

    for _ in range(epochs):
        y_hat = Xb@w
        err = y_hat - y
        loss = (err @ err) / (2 * n)

        grad = (Xb.T @ err)/ n
        w_new = w - lr * grad
        if np.linalg.norm(w_new - w) < tol:
            w = w_new
            break
        w = w_new
    return w, history

In [4]:
import torch

def gd_linear_regression_torch(X, y, lr=1e-2, epochs=1000, tol=1e-6, device="cpu"):
    X = torch.as_tensor(X, dtype=torch.float32, device=device)   # (n, d)
    y = torch.as_tensor(y, dtype=torch.float32, device=device).view(-1, 1)  # (n, 1)

    n, d = X.shape
    W = torch.zeros(d, 1, device=device, requires_grad=True)     # weights
    b = torch.zeros(1, device=device, requires_grad=True)        # bias

    history = []
    for _ in range(epochs):
        y_hat = X @ W + b                                        # (n,1)
        loss = ((y_hat - y).pow(2).mean()) / 2                   # MSE/2
        history.append(loss.item())

        loss.backward()                                          # compute grads
        with torch.no_grad():                                    # update params
            W -= lr * W.grad
            b -= lr * b.grad
            # simple convergence check on parameter delta
            if (W.grad.norm() * lr + b.grad.abs().sum() * lr) < tol:
                pass
            W.grad.zero_(); b.grad.zero_()                       # clear grads
    return W.detach().cpu().numpy().ravel(), b.item(), history

ModuleNotFoundError: No module named 'torch'

In [ ]:
import torch
import torch.nn as nn

class LinearReg(nn.Module):
    def __init__(self, d):
        super().__init__()
        self.fc = nn.Linear(d, 1)  # includes weight and bias
    def forward(self, X):
        return self.fc(X)

def train_lr(X, y, lr=1e-2, epochs=1000, device="cpu"):
    X = torch.as_tensor(X, dtype=torch.float32, device=device)
    y = torch.as_tensor(y, dtype=torch.float32, device=device).view(-1, 1)

    model = LinearReg(X.shape[1]).to(device)
    criterion = nn.MSELoss(reduction="mean")                     # PyTorch's MSE
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)

    history = []
    for _ in range(epochs):
        optimizer.zero_grad()                                    # clear old grads
        y_hat = model(X)
        loss = criterion(y_hat, y) / 2                           # (optional) /2
        history.append(loss.item())
        loss.backward()                                          # autograd
        optimizer.step()                                         # GD update
    w = model.fc.weight.detach().cpu().numpy().ravel()
    b = model.fc.bias.detach().cpu().item()
    return w, b, history

## Backpropogation

1) **Linear regression (the simplest backprop)**

**Model**  
$$
\hat{y} = XW + b
$$

**Loss (MSE)**  
$$
L = \frac{1}{N}\sum_{i=1}^{N}\lVert \hat{y}_i - y_i \rVert^2
$$

**Gradients**  
$$
\nabla_{\hat{y}} L = \frac{2}{N}\,(\hat{y} - y)
$$

$$
\nabla_W L = X^\top \,\nabla_{\hat{y}} L
$$

$$
\nabla_b L = \sum_{i=1}^{N} \left(\nabla_{\hat{y}} L\right)_i
$$



In [5]:
import numpy as np

np.random.seed(0)
N, D, O = 128, 3, 1          # samples, input dim, output dim
X = np.random.randn(N, D)
true_W = np.array([[2.0],[ -3.0],[1.5]])
true_b = np.array([0.7])
y = X @ true_W + true_b + 0.1*np.random.randn(N, O)

W = np.random.randn(D, O) * 0.01
b = np.zeros(O)
lr = 1e-2

for _ in range(1000):
    # forward
    y_hat = X @ W + b         # (N,O)
    loss = np.mean((y_hat - y)**2)

    # backward
    dYhat = (2.0 / N) * (y_hat - y)     # (N,O)
    dW = X.T @ dYhat                     # (D,O)
    db = dYhat.sum(axis=0)               # (O,)

    # update
    W -= lr * dW
    b -= lr * db

print("W:", W.ravel(), "b:", b)

W: [ 2.00512262 -3.00671982  1.4890194 ] b: [0.69656805]


## 2) Two-layer NN (ReLU -> Linear -> Softmax) — Backprop

**Architecture**
$$
\begin{aligned}
Z_1 &= X W_1 + \mathbf{1} b_1^{\top} \\
h  &= \mathrm{ReLU}(Z_1) \\
\text{scores} &= h W_2 + \mathbf{1} b_2^{\top} \\
\text{probs}_{i,j} &= \frac{\exp(\text{scores}_{i,j})}{\sum_{c=1}^{C} \exp(\text{scores}_{i,c})}
\end{aligned}
$$

**Cross-entropy loss (labels as class indices \(y_i\))**
$$
L \;=\; -\frac{1}{N}\sum_{i=1}^{N}\log\!\big(\text{probs}_{i,\,y_i}\big)
$$

**Softmax + CE gradient (vectorized)**
Let $Y \in \mathbb{R}^{N \times C}$ be one-hot labels. Then
$$
G \;=\; \frac{1}{N}\,\big(\text{probs} - Y\big)
$$

**Top linear layer**
$$
\nabla_{W_2} L \;=\; h^{\top} G, 
\qquad
\nabla_{b_2} L \;=\; \sum_{i=1}^{N} G_{i,\cdot},
\qquad
\nabla_{h} L \;=\; G W_2^{\top}
$$

**ReLU backward**
Let $Z_1 = X W_1 + \mathbf{1}\, b_1^{\top}$. Then
$$
\nabla_{h}^{(\text{after})} \;=\; \nabla_{h}^{(\text{before})} \;\odot\; \mathbb{I}[\,Z_1 > 0\,]
$$

**Bottom linear layer**
$$
\nabla_{W_1} L \;=\; X^{\top}\,\nabla_{h}^{(\text{after})},
\qquad
\nabla_{b_1} L \;=\; \sum_{i=1}^{N} \nabla_{h,i}^{(\text{after})}
$$

**Shape reference**
$$
\begin{aligned}
&X\in\mathbb{R}^{N\times D},\quad W_1\in\mathbb{R}^{D\times H},\quad b_1\in\mathbb{R}^{H} \\
&h\in\mathbb{R}^{N\times H},\quad W_2\in\mathbb{R}^{H\times C},\quad b_2\in\mathbb{R}^{C} \\
&\text{scores},\,\text{probs},\,Y,\,G\in\mathbb{R}^{N\times C}
\end{aligned}
$$


In [12]:
import numpy as np

np.random.seed(1)
N, D, H, C = 200, 4, 16, 3        # samples, input dim, hidden, classes
X = np.random.randn(N, D)
y = np.random.randint(0, C, size=N)

W1 = 0.01*np.random.randn(D, H); b1 = np.zeros(H)
W2 = 0.01*np.random.randn(H, C); b2 = np.zeros(C)

lr = 1e-0

for _ in range(1000):
    # ----- forward -----
    z1 = X @ W1 + b1           # (N,H)
    h = np.maximum(0, z1)      # ReLU (N,H)
    scores = h @ W2 + b2       # (N,C)

    # softmax (stable)
    scores_shift = scores - scores.max(axis=1, keepdims=True)
    exp_scores = np.exp(scores_shift)
    probs = exp_scores / exp_scores.sum(axis=1, keepdims=True)   # (N,C)

    # cross-entropy loss
    correct_logprobs = -np.log(probs[np.arange(N), y] + 1e-12)
    ## Binary case
    # loss = -np.mean(y * np.log(probs + 1e-12) + (1 - y) * np.log(1 - probs + 1e-12))
    loss = correct_logprobs.mean()

    # ----- backward -----
    dscores = probs.copy()                    # (N,C)
    dscores[np.arange(N), y] -= 1
    dscores /= N

    dW2 = h.T @ dscores                      # (H,C)
    db2 = dscores.sum(axis=0)                # (C,)

    dh = dscores @ W2.T                      # (N,H)
    dh[z1 <= 0] = 0                          # ReLU backprop

    dW1 = X.T @ dh                           # (D,H)
    db1 = dh.sum(axis=0)                     # (H,)

    # ----- update -----
    W1 -= lr * dW1; b1 -= lr * db1
    W2 -= lr * dW2; b2 -= lr * db2

# quick sanity check: accuracy
scores = np.maximum(0, X @ W1 + b1) @ W2 + b2
pred = scores.argmax(axis=1)
acc = (pred == y).mean()
print(f"loss={loss:.4f}, acc={acc:.3f}")


loss=0.9784, acc=0.490


## Attention Layer

In [ ]:
import torch
import torch.nn as nn


class MultiHeadSelfAttention(nn.Module):
    def __init__(self, embed_size: int, num_heads: int):
        ## Identical to super().__init__
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_size = embed_size
        self.num_heads = num_heads
        self.head_dim = embed_size // num_heads

        assert (
            self.head_dim * num_heads == embed_size
        ), "Embedding size must be divisible by the number of heads."

        self.values = nn.Linear(embed_size, embed_size, bias=False)
        self.keys = nn.Linear(embed_size, embed_size, bias=False)
        self.queries = nn.Linear(embed_size, embed_size, bias=False)
        self.fc_out = nn.Linear(embed_size, embed_size)

    def forward(self, values: torch.Tensor, keys: torch.Tensor, queries: torch.Tensor, mask: torch.Tensor = None):
        batch_size = queries.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], queries.shape[1]

        # Split the embedding into self.num_heads different pieces
        values = self.values(values).reshape(batch_size, value_len, self.num_heads, self.head_dim)
        keys = self.keys(keys).reshape(batch_size, key_len, self.num_heads, self.head_dim)
        queries = self.queries(queries).reshape(batch_size, query_len, self.num_heads, self.head_dim)

        # Transpose for attention dot product: (batch_size, num_heads, seq_len, head_dim)
        values = values.transpose(1, 2)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)

        # Scaled dot-product attention
        ## Before queries = [B, num_heads, query_len, head_dim]
        ##        keys    = [B, num_heads, key_len, head_dim]
        ## keys.transpose(-2, -1)) swaps the last 2 dimensions ==> (B, H, N, L)
        energy = torch.matmul(queries, keys.transpose(-2, -1)) / (self.head_dim ** 0.5)

        ## Mask usually has shape [batch_size, key_len]
        ## Unsqueeze make it [B, 1, 1, K_len]
        if mask is not None:
            # Expand mask to match energy shape
            mask = mask.unsqueeze(1).unsqueeze(2)
            mask = mask.expand(-1, self.num_heads, query_len, -1)
            energy = energy.masked_fill(mask == 0, -float("Inf"))

        attention = torch.softmax(energy, dim=-1)

        out = torch.matmul(attention, values)  # (batch_size, num_heads, query_len, head_dim)
        ## forces PyTorch to actually copy the tensor into a new block of memory laid out in row-major (C-style) order.
        out = out.transpose(1, 2).contiguous()  # (batch_size, query_len, num_heads, head_dim)
        out = out.reshape(batch_size, query_len, self.embed_size)  # Concatenate heads

        out = self.fc_out(out)  # Apply the final linear layer
        return out

In [ ]:
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, embed_size: int, num_heads: int):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_size = embed_size
        self.num_heads = num_heads
        self.head_dim = embed_size // num_heads

        assert (
            self.head_dim * num_heads == embed_size
        ), "Embedding size must be divisible by the number of heads."

        self.values = nn.Linear(embed_size, embed_size, bias=False)
        self.keys = nn.Linear(embed_size, embed_size, bias=False)
        self.queries = nn.Linear(embed_size, embed_size, bias=False)
        self.fc_out = nn.Linear(embed_size, embed_size)

        # 5000 is the max relative distance
        self.relative_position_embeddings = nn.Parameter(
            torch.randn((2 * 5000 - 1, self.head_dim))
        )

    def forward(self, values: torch.Tensor, keys: torch.Tensor, queries: torch.Tensor, mask: torch.Tensor = None):
        batch_size = queries.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], queries.shape[1]

        values = self.values(values).reshape(batch_size, value_len, self.num_heads, self.head_dim)
        keys = self.keys(keys).reshape(batch_size, key_len, self.num_heads, self.head_dim)
        queries = self.queries(queries).reshape(batch_size, query_len, self.num_heads, self.head_dim)

        values = values.transpose(1, 2)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)

        ## Trick to use broadcast
        relative_positions = torch.arange(query_len).unsqueeze(1) - torch.arange(key_len).unsqueeze(0)
        # Example:
        # tensor([[2, 1, 0, 0],
        #         [3, 2, 1, 0],
        #         [4, 3, 2, 1],
        #         [4, 4, 3, 2]])
        clipped_positions = torch.clamp(relative_positions + 5000 - 1, min=0, max=2 * 5000 - 2)
        ## E∈R(2L−1)×dh look up table
        relative_embeddings = self.relative_position_embeddings[clipped_positions]  # (query_len, key_len, head_dim)

        # Scaled dot-product attention with relative positional embeddings
        energy = torch.matmul(queries, keys.transpose(-2, -1)) / (self.head_dim ** 0.5
        # for each position q, we extract k of d dimensional position verctor
        # You give it a string equation that describes how indices line up, and PyTorch does the math automatically.
        energy += torch.einsum('bhqd,qkd->bhqk', queries, relative_embeddings)  # Add relative positional embeddings

        if mask is not None:
            # Expand mask to match energy shape
            mask = mask.unsqueeze(1).unsqueeze(2)
            mask = mask.expand(-1, self.num_heads, query_len, -1)
            energy = energy.masked_fill(mask == 0, -float("Inf"))

        attention = torch.softmax(energy, dim=-1)

        out = torch.matmul(attention, values)
        out = out.transpose(1, 2).contiguous()
        out = out.reshape(batch_size, query_len, self.embed_size)

        out = self.fc_out(out)
        return out


## Byte Pair Encoding

In [15]:
"hello 😊" → "😊" becomes <unk>

SyntaxError: invalid character '→' (U+2192) (2483536979.py, line 1)

The tokenizer only knows merge rules learned during training.
If a new word contains a character sequence that never appeared during training,
the tokenizer doesn’t know how to split it.

In [ ]:
from collections import defaultdict
from typing import List, Tuple


class BPE:
    def __init__(self, vocab_size: int, min_frequency: int = 2):
        """
        Initialize the BPE tokenizer with the desired vocabulary size and minimum frequency for pair merges.
        """
        self.vocab_size = vocab_size
        self.min_frequency = min_frequency
        self.vocab = {}
        self.merges = {}

    def get_vocab(self):
        return self.vocab

    def train(self, corpus: List[str]) -> None:
        """
        Train the BPE tokenizer on the given corpus.
        """
        # Build initial word frequency and split words into characters
        word_freqs = defaultdict(int)
        for text in corpus:
            words = text.split()
            for word in words:
                word = word + ' '  # Add a space at the end of each word to preserve word boundaries
                word_freqs[word] += 1
        ## word_freqs = {"the ": 2, "cat ": 1, "dog ": 1}

        # Initialize splits: each word is split into a list of characters
        splits = {word: list(word) for word in word_freqs}

        # splits = {
        #   "the ": ["t", "h", "e", " "],
        #   "cat ": ["c", "a", "t", " "],
        #   "dog ": ["d", "o", "g", " "]
        # }

        # Build vocabulary with single characters
        alphabet = set()
        for word in splits:
            for char in splits[word]:
                alphabet.add(char)

        ## alphabet = {"t", "h", "e", " ", "c", "a", "d", "o", "g"}

        # Initialize vocabulary with single characters
        self.vocab = {char: idx for idx, char in enumerate(alphabet)}

        # self.vocab = {
        #   "t": 0,
        #   "h": 1,
        #   "e": 2,
        #   " ": 3,
        #   "c": 4,
        #   "a": 5,
        #   "d": 6,
        #   "o": 7,
        #   "g": 8
        # }

        # Continue merging pairs until the vocabulary reaches the desired size
        # After merging "t" and "h", we have "t", "h" and "th"
        # {
        #   ('t', 'h'): 2,
        #   ('h', 'e'): 2,
        #   ('e', ' '): 2,
        #   ('c', 'a'): 1,
        #   ('a', 't'): 1,
        #   ('t', ' '): 1,
        #   ('d', 'o'): 1,
        #   ('o', 'g'): 1,
        #   ('g', ' '): 1
        # }
        while len(self.vocab) < self.vocab_size:
            pair_freqs = self._get_pair_frequencies(splits, word_freqs)

            if not pair_freqs:
                break

            # Get the most frequent pair of symbols
            best_pair, best_freq = max(pair_freqs.items(), key=lambda x: x[1])

            if best_freq < self.min_frequency:
                break

            # Merge the most frequent pair
            splits = self._merge_pair(best_pair, splits)

            # splits = {
            #   "the ": ["th", "e", " "],
            #   "cat ": ["c", "a", "t", " "],
            #   "dog ": ["d", "o", "g", " "]
            # }
            
            # Add the new merge to the vocabulary and merges table
            new_symbol = best_pair[0] + best_pair[1]
            ## "th"
            self.vocab[new_symbol] = len(self.vocab)
            self.merges[best_pair] = new_symbol

    def encode(self, text: str) -> List[int]:
        """
        Encode the input text into a list of token ids.
        """
        words = text.split()
        tokens = []

        for word in words:
            word = word + ' '  # Add a space at the end of each word to preserve word boundaries
            split_word = list(word)
            for pair, merge in self.merges.items():
                i = 0
                while i < len(split_word) - 1:
                    if split_word[i] == pair[0] and split_word[i + 1] == pair[1]:
                        split_word = split_word[:i] + [merge] + split_word[i + 2:]
                    else:
                        i += 1

            # Convert merged word parts to token ids
            for token in split_word:
                tokens.append(self.vocab.get(token, -1))  # -1 for unknown tokens

        return tokens

    def decode(self, tokens: List[int]) -> str:
        """
        Decode the list of token ids back into text.
        """
        ## tokens = [11, 4, 5, 0, 3]
        ## ["the ", "c", "a", "t", " "]
        # Inverse the vocabulary dictionary to get symbol to token mapping
        inv_vocab = {v: k for k, v in self.vocab.items()}
        decoded_text = []

        for token in tokens:
            if token in inv_vocab:
                decoded_text.append(inv_vocab[token])
            else:
                decoded_text.append('<UNK>')  # Handle unknown tokens

        # Join the decoded text and strip the trailing spaces after joining
        return ''.join(decoded_text).strip()

    def _get_pair_frequencies(self, splits: dict, word_freqs: dict) -> dict:
        """
        Get the frequency of all symbol pairs in the current split words.
        """
        pair_freqs = defaultdict(int)
        for word, freq in word_freqs.items():
            split = splits[word]
            if len(split) == 1:
                continue
            for i in range(len(split) - 1):
                pair = (split[i], split[i + 1])
                pair_freqs[pair] += freq
        return pair_freqs

    def _merge_pair(self, pair: Tuple[str, str], splits: dict) -> dict:
        """
        Merge the given pair in all the words.
        """
        for word in splits:
            split = splits[word]
            i = 0
            while i < len(split) - 1:
                if split[i] == pair[0] and split[i + 1] == pair[1]:
                    split = split[:i] + [pair[0] + pair[1]] + split[i + 2:]
                else:
                    i += 1
            splits[word] = split
        return splits
